In [40]:
import pandas as pd
import numpy as np
import matplotlib as plt
import requests
import import_ipynb

from keras.models import load_model

In [41]:
api_key = open('./../api_key.txt', 'r').read()

url = 'https://api.binance.com/api/v3/klines'

params = {
    'symbol':  'BTCUSDT',
    'interval': '1h',
    'limit': 720
}
headers = {
    'X-MBX-APIKEY': api_key
}

# Get current date, 4 years ago


results = requests.get(url, params=params)

In [42]:
data = pd.DataFrame(results.json())


In [43]:
data

,0,1,2,3,4,5,6,7,8,9,10,11
0,1679774400000,27189.69000000,27430.14000000,27156.09000000,27354.72000000,3202.60967000,1679777999999,87465458.62413750,61874,1782.91618000,48690279.95346410,0
1,1679778000000,27354.73000000,27455.00000000,27349.09000000,27411.54000000,1058.74562000,1679781599999,29010399.60546980,32555,561.03097000,15372298.21099610,0
2,1679781600000,27411.54000000,27529.29000000,27391.05000000,27447.32000000,1518.28385000,1679785199999,41703535.98966580,39261,769.61206000,21138243.42046260,0
3,1679785200000,27447.31000000,27486.50000000,27404.99000000,27462.95000000,808.11091000,1679788799999,22186252.25788360,24027,403.03372000,11064992.27027840,0
4,1679788800000,27462.96000000,27630.32000000,27417.76000000,27531.28000000,1997.79617000,1679792399999,54989449.73966200,51053,1034.97871000,28488408.98451610,0
...,...,...,...,...,...,...,...,...,...,...,...,...
715,1682348400000,27430.19000000,27431.78000000,27200.00000000,27267.05000000,2847.37861000,1682351999999,77728041.78680620,70232,1311.27382000,35796569.97795140,0
716,1682352000000,27267.06000000,27394.52000000,27028.64000000,27110.55000000,3676.58845000,1682355599999,100116369.40284810,77477,1883.27479000,51287983.74345380,0
717,1682355600000,27110.56000000,27409.46000000,26942.82000000,27381.49000000,4336.61496000,1682359199999,118079924.07013030,85391,2041.30660000,55595690.85894180,0
718,1682359200000,27381.48000000,27579.80000000,27290.00000000,27339.90000000,2879.10149000,1682362799999,78930138.36808090,59206,1370.20721000,37557375.30679880,0


In [44]:
data = data[1]

In [45]:
data

0      27189.69000000
1      27354.73000000
2      27411.54000000
3      27447.31000000
4      27462.96000000
            ...      
715    27430.19000000
716    27267.06000000
717    27110.56000000
718    27381.48000000
719    27339.91000000
Name: 1, Length: 720, dtype: object

In [46]:
train_min = float(open('./../train_min_max_std/min.txt', 'r').read())
train_max = float(open('./../train_min_max_std/max.txt', 'r').read())

In [47]:
type(train_min), type(train_max)

(float, float)

In [48]:
data_norm = (data.astype(float) - train_min) / (train_max - train_min)

In [49]:
data_norm = np.array(data_norm)
data_norm = data_norm.reshape(1, 720, 1)
data_norm

array([[[0.35747394],
        [0.36003255],
        [0.36091327],
        [0.36146781],
        [0.36171043],
        [0.36276959],
        [0.36373171],
        [0.36367419],
        [0.36273843],
        [0.36254232],
        [0.36297935],
        [0.36223304],
        [0.36236125],
        [0.36475273],
        [0.36517968],
        [0.3650538 ],
        [0.36795812],
        [0.36879171],
        [0.37236157],
        [0.36686144],
        [0.36699585],
        [0.36537393],
        [0.36752171],
        [0.3670712 ],
        [0.36736885],
        [0.36658874],
        [0.36752621],
        [0.37066059],
        [0.36954081],
        [0.3699473 ],
        [0.36735381],
        [0.36816276],
        [0.36831035],
        [0.36859405],
        [0.36562074],
        [0.36671633],
        [0.36707197],
        [0.3676879 ],
        [0.36888333],
        [0.36872427],
        [0.36837251],
        [0.36616024],
        [0.36527673],
        [0.35391897],
        [0.35303917],
        [0

In [50]:
model = load_model('saved_model.h5')

In [51]:
prediction_norm = model.predict(data_norm)
prediction_norm

1/1 [==============================] - 1s 870ms/step


array([[0.3589742 , 0.36137736, 0.36005098, 0.356635  , 0.35573128,
        0.3601957 , 0.3599288 , 0.35702115]], dtype=float32)

In [52]:
pd.DataFrame(prediction_norm)

,0,1,2,3,4,5,6,7
0,0.358974,0.361377,0.360051,0.356635,0.355731,0.360196,0.359929,0.357021


In [53]:
prediction = (prediction_norm * (train_max - train_min) + train_min)

In [54]:
prediction

array([[27286.463, 27441.477, 27355.92 , 27135.576, 27077.281, 27365.254,
        27348.037, 27160.484]], dtype=float32)

In [55]:
print(f'Next hour prediction: {prediction[0][0]}')

Next hour prediction: 27286.462890625
